# T4 - IIC2613 | Inteligencia Artificial
## Aprendizaje de Máquina

Fecha de entrega: Jueves 13 de Junio de 2024 a las 23:59 hrs.

*Recuerda que todas tus resupuestas (tanto teoricas como de codigo) deben ir en este notebook, el cual debe ser subido a tu repositorio de GitHub antes de la fecha de entrega.*

Las respuestas teoricas deberan estar completadas en celdas de Markdown debajo de su pregunta respectiva, ademas, se solicita que todas las celdas del notebook se encuentren ejecutadas a la hora de su entrega.

---

# Parte 1: Reflexión (2 pts.)

Esta sección de la tarea consiste en una serie de preguntas de reflexión que tendrás que responder basándote en el podcast de la profesora Jocelyn Dunstan ["Ciencia de Datos con Jocelyn Dunstan"](https://open.spotify.com/show/4h0tWwoeer64BAz2mfW2pt?si=d2db3ab40ebc4fd2) que está disponible en Spotify. Escucha uno a mas capítulos del podcast y escoge un episodio para contestar las siguientes preguntas:

<p align="center">
    <img src="https://i.scdn.co/image/ab6765630000ba8a48be458b1c3392d7c716d150" alt="Logo del Podcast 'Ciencia de Datos' de Jocelyn Dunstan" width="20%">
</p>

### 1.1 ¿Por qué escogiste este episodio?

### 1.2 ¿Cuál crees que es la relevancia del tema tratado en el capítulo dentro del campo de la inteligencia artificial, machine learning o ciencia de datos?

### 1.3 Busca al menos una referencia externa que enriquezca alguna idea mencionada en el capítulo y discute.

---

# Parte 2: DCClasificador de Sismos (4 pts)

El objetivo de esta actividad es entrenar algoritmos de clasificacion y regresion para estimar la posibilidad de tsunami y magnitud en grados richter de un sismo dada informacion sobre este.

Para llevar esto a cabo se dispone de una base de datos que contiene todos los sismos entre los años 1990 y 2023 (mas de 3.5M de muestras), de la cual se ha tomado un segmento de ~10.000 muestras para poder implementar algoritmos de aprendizaje supervisado. El set de datos original se encuentra disponible en [Kaggle](https://www.kaggle.com/datasets/alessandrolobello/the-ultimate-earthquake-dataset-from-1990-2023), junto con el detalle sobre sus contenidos.

## Importar librerías
Recuerda haber instalado previamente las librerias `numpy`, `pandas`, `scikit-learn`, `matplotlib` y `seaborn`.

In [1]:
# Las siguientes librerias nos permiten el trabajo de los datos
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Las siguientes librerias nos permiten la visualización de los datos
import seaborn as sns
import matplotlib.pyplot as plt

# Las siguientes funciones nos permiten medir el desempeño de los modelos
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# Importamos las clases para clasificación
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Importamos las clases para regresión
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

## 2.1 Lectura y estudio de los datos

### 2.1.1 Carga de los datos

Carga los datos del archivo `earthquakes.csv` utilizando la librería de procesamiento de datos [pandas](https://pandas.pydata.org/docs/). Comenta sobre los elementos que contiene, para esto debes investigar la fuente de los datos y visualizarlos con pandas para su entendimiento.

### 2.1.2 Descripcion de los datos

Utiliza los metodos [`describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) y [`count()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.count.html) de la clase `DataFrame` de Pandas para realizar un análisis inicial de los datos. Indica la cantidad de datos con los que se está trabajando, el tipo de datos presentes y cualquier otra información que consideres relevante. 

Una de las actividades en esta tarea consiste en predecir la posibilidad de un tsunami dada el resto de la informacion recompilada de un sismo, indica que columna usaremos de etiqueta y por qué es importante mantenerla oculta al momento de testear la clasificación. 

### 2.1.3 Desbalance de clases

Investiga sobre qué es el balance/desbalance de clases, cómo puede afectar este a un algoritmo clasificador e indica alguna tecnica para contrarrestar el desbalance en problemas de clasificacion. Finalmente, indica la distribución de datos por clase, si están balanceados o no y cómo esto podría beneficiar/afectar al rendimiento de los modelos que entrenaremos en esta tarea.

(*Hint: Usar el metodo [`value_counts()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html) para una columna de un DataFrame nos retorna el numero de instancias por clase, tambien se recomienda graficar [el histograma de frecuencias por valor](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html)*)

## 2.2 Preprocesamiento de datos

En esta sección, deberás llevar a cabo un tratamiento de los datos para garantizar que los clasificadores puedan ser entrenados de manera adecuada. Es importante resaltar que todas las decisiones que tomes deberán estar debidamente justificadas. Para comenzar responde a las preguntas:

¿Por qué es importante preprocesar los datos y cómo puede esto afectar a nuestro problema de clasificación?

### 2.2.1 Muestreo de datos

Los datos cargados corresponden a un segmento de un [set de datos mas grande sobre terremotos a lo largo del mundo](https://www.kaggle.com/datasets/alessandrolobello/the-ultimate-earthquake-dataset-from-1990-2023), el set de datos original tiene 3.5 millones de filas pero solamente unas 10.000 fueron extraidas, guardando todas las muestras con tsunami (debido a lo poco frecuente de la clase) y tratando de balancear la magnitud de un terremoto para no tener sobrerepresentacion de algunos valores. A continuacion se muestra un grafico de la distribucion (normalizada) de la columna `magnitude` antes y despues de muestrear el set original:

<p align="center">
    <img src="https://gcdnb.pbrd.co/images/3nA2jNg0zXv9.png?o=1" alt="Distribucion de la columna magnitud en el set de datos original vs la muestra" width="25%">
</p>

En caso de que hubiesemos dispuesto de pocos datos originalmente ¿qué técnicas podrías usar para enriquecer el dataset? Investiga en internet y reporta al menos dos metodologias.

### 2.2.2 Valores nulos

Revisa si el nuevo DataFrame contiene valores nulos ([NaN](https://pandas.pydata.org/docs/reference/api/pandas.isna.html)). En caso afirmativo, deberás tomar la decisión de eliminar las filas o columnas que contengan estos datos nulos, o si prefieres, reemplazar estos con algún otro valor. Si no existen valores nulos, indícalo. ¿Por qué es importante manejar este tipo de valores?.

### 2.2.3 Variables categoricas y encoding

¿Qué son las variables categóricas? ¿Hay este tipo de variables en este conjunto de datos? Explica cómo se pueden manejar estas y en caso de haberlas realiza un desarrollo para poder entrenar los algoritmos correctamente.

Se recomienda **fuertemente** revisar [el siguiente link](https://medium.com/geekculture/machine-learning-one-hot-encoding-vs-integer-encoding-f180eb831cf1) sobre metodos de encoding.

### 2.2.4 Limpieza de los datos

Elimina las columnas que no consideres necesarias o que creas que no aportan información relevante para realizar la clasificación. Debes justificar por qué eliminas cada columna. Responde a la pregunta ¿Cómo pueden afectar columnas con información no relevante al modelo de clasificación?

Se recomienda **fuertemente** usar la funcion [`pairplot()`](https://seaborn.pydata.org/generated/seaborn.pairplot.html) de [Seaborn](https://seaborn.pydata.org/index.html) para visualizar la correlacion entre caracteristicas, aunque recuerda que **correlacion no siempre implica causalidad**.

### 2.2.5 Separacion en conjuntos de entrenamiento/testeo

Nos importa que tanto nuestro set de entrenamiento como nuestro set de testeo sea igual de representativos de la distribucion de los datos, por lo que tomaremos en consideracion esto a la hora de separar los datos en train/test. Afortunadamente, la funcion [`train_test_split()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de [SciKit Learn](https://scikit-learn.org/stable/index.html) nos permite llevar esto a cabo mediante su atributo `stratify`. Usaremos un 20% de los datos para evaluar nuestros modelos.

Dado que queremos que ambos set de datos contengan la misma proporcion de datos de la columna `tsunami`, la pasaremos de argumento para llevar a cabo la separacion, como se muestra a continuacion:

In [6]:
# OJO: Debes reemplazar la variable 'df' por el nombre de la variable que contiene tu dataset
X_train, X_test = train_test_split(df, test_size = 0.2, random_state = 42, stratify = df['tsunami'])

### 2.2.6 Definicion de etiquetas

Como fue comentado anteriormente, dentro de esta tarea llevaremos a cabo dos tareas sobre el set de datos, clasificacion de tsunamis y regresion de la magnitud de un sismo utilizando informacion externa sobre este. Por tanto, deberas separar las columnas `tsunami` y `magnitude` del set de datos para utilizarlas como vectores de etiquetas $\vec{y}$ para tus clasificadores.

Dentro de este apartado se espera que tengas 2 matrices y 4 vectores:
- `X_train` y `X_test`: Matrices que contienen la informacion sobre cada sismo (excepto las columnas `tsunami` y `magnitude`).
- `y_train_tsunami` y `y_test_tsunami`: Vectores de entrenamiento y evaluacion que contienen un 0 para sismos sin tsunamis y un 1 para aquellos que si.
- `y_train_magnitude` y `y_test_magnitude`: Vectores de entrenamiento y evaluacion que contienen la magnitud para cada sismo dentro de `X_train` y `X_test` respectivamente.

### 2.2.7 Normalizacion de los datos

¿Qué es la normalización de datos y por qué puede ser beneficioso realizarla para un problema de clasificación? ¿Existe alguna columna que pueda ser conveniente normalizar? En caso de haberla utiliza algún algoritmo de normalización, si no justifica por qué no es conveniente realizar dicha operación.

Es importante que la obtencion de parametros para normalizacion de los datos se lleva a cabo en el conjunto de entrenamiento y luego se aplica sobre el de evaluacion, si utilizas [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) de sklearn, deberas hacer ajuste del scaler (usando `.fit()`) sobre la columna en el conjunto de entrenamiento y luego hacer `.transform()` sobre ambos conjuntos.

## 2.3 Explorando los algoritmos de un problema de clasificación

Responde las siguientes preguntas de análisis para un problema de clasificación y los algoritmos que se utilizan para estos:

### 2.3.1 Nociones basicas
¿Cuál es el principal objetivo de un problema de clasificación?

### 2.3.2 Ejemplos practicos
¿Qué casos de uso comunes de clasificación existen en el mundo real? ¿Qué impacto pueden tener estos en las personas? Menciona al menos 3.

### 2.3.3 Desafios e impacto
¿Qué relevancia e impacto puede tener el éxito o fracaso de este problema de clasificación en la vida real? ¿De qué modo un algoritmo de clasificación como este podría beneficiar a la sociedad chilena para prepararse ante desastres naturales?

### 2.3.4 Seleccion de algoritmos
¿Cuáles son los factores que deberían influir en la elección de un algoritmo de clasificación para un conjunto de datos determinado?

### 2.3.5 Definicion de hiperparametros
¿Qué son los hiperparámetros y cómo estos podrían afectar al rendimiento de un modelo de clasificación?

## 2.4 Implementación de Clasificadores

Utilizando el DataFrame con los datos ya preprocesados, deberás entrenar cuatro modelos distintos. Para esto, se deben entrenar los clasificadores de KNN, árboles de decisión, Random Forest y SVM utilizando sus implementaciones de sklearn.

*En esta seccion solo debes hacer el ajuste de los modelos, el calculo de metricas de desempeño sera llevado a cabo en el apartado siguiente.* 

### 2.4.1 KNN (K-Nearest Neighbors):

### 2.4.2 Decision Tree

### 2.4.3 Random Forest

### 2.4.4 SVM

## 2.5 Cálculo de métricas

### 2.5.1 Metricas basicas de rendimiento
Genera una tabla comparativa en la que se indiquen las métricas de accuracy, precisión y recall para cada uno de los modelos.

### 2.5.2 F1-Score
Investiga qué es la métrica F1-score y calcúlala a partir de los resultados, no puedes usar implementaciones ya existentes para calcularlo. Luego argumenta, ¿qué mide el score F1 en problemas de clasificacion binaria?

### 2.5.3 Visualizacion de resultados
Grafica utilizando la librería matplotlib el F1-score para cada modelo (con un barplot esta bien).

### 2.5.4 Modelo de mejor desempeño
Señala cuál de los modelos obtuvo las mejores métricas y rendimiento.

### 2.5.5 Ajuste de hiperparametros
Para el algoritmo con los mejores resultados elige 2 hiperparámetros y entrena 3 modelos variando estos, con este el mismo algoritmo, realiza predicciones con este y luego evalúalos. ¿Qué combinación de hiperparámetros es la con mejores resultados?

### 2.5.6 Matriz de confusion
Para el modelo de mejor desempeño del apartado anterior calcula su [matriz de confusion](https://datascientest.com/es/matriz-de-confusion) utilizando el metodo de sklearn `confusion_matrix()` y visualizala utilizando [`ConfusionMatrixDisplay`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html). Luego comenta, ¿en qué tiende a equivocarse el modelo entrenado?

## 2.6 Análisis de desempeño y resultados

### 2.6.1 Analisis de modelos
Explica por qué crees que algunos algoritmos tuvieron mejores resultados que otros para este set de datos. Basa tu respuesta en material bibliográfico, ¿significa eso que algunos algoritmos tienen siempre mejores resultados que otros?

### 2.6.2 Mejora de desempeño
¿Qué medidas se podrían tomar para mejorar el rendimiento de los modelos?

### 2.6.3 Importancia de metricas
¿Por qué son tan importantes las métricas estadisticas para los modelos y/o clasificadores realizados anteriormente?

### 2.6.4 Utilidad del modelo
¿Qué utilidad le vas al uso de tecnicas como esta en la vida real para tener un impacto positivo en la sociedad? ¿Qué tipo de decisiones se podrían tomar o qué medidas se podrían implementar en base a las predicciones de un algoritmo como este?

## 2.7 Implementación de Clasificadores

Utilizando el DataFrame con los datos ya preprocesados, entrena cuatro regresores distintos para predecir la magnitud de los sismos a partir de otras características. Con los datos de entrenamiento, realiza una predicción de regresión con los modelos de KNN, árboles de decisión, Random Forest y SVM.

*En esta seccion solo debes hacer el ajuste de los modelos, el calculo de metricas de desempeño sera llevado a cabo en el apartado siguiente.* 

### 2.7.1 KNN (K-Nearest Neighbors):

### 2.7.2 Decision Tree

### 2.7.3 Random Forest

### 2.7.4 SVM

## 2.8 Análisis de regresión

### 2.8.1 Root Mean Squared Error
Para esta parte deberas calcular, imprimir y generar un gráfico en el cual se indiquen los RMSE (Root Mean Squared Error) para cada regresión sobre los datos del set de testing.

### 2.9.2 Implicancias del RMSE
¿Qué significa tener un mayor o menor RMSE y que implicancias tienen en la clasificación y sus resultados?

### 2.9.3 Comparacion de modelos
Al realizar la comparación entre cada modelo de regresión y su RMSE, detalle la razón de la obtención de estos resultados específicos, el porque un modelo de regresión es mejor que otro dada su construcción y explicar en tus propias palabras el funcionamiento de estos modelos de regresión.